In [ ]:
# Retrieve the data from the SQLite database
query = "SELECT * FROM tweets"
tweets_df = pd.read_sql_query(query, conn, parse_dates=['created_at'])
print(tweets_df.dtypes)


# Retrieve the data from the SQLite database
query = "SELECT * FROM users"
users_df = pd.read_sql_query(query, conn, parse_dates=['author_created'])
print(users_df.dtypes)

In [ ]:
def aggregate_data(df):
    """
    Aggregate data based on domain ID from the provided tweets DataFrame.

    Args:
        tweets_df (pd.DataFrame): DataFrame containing tweet data.

    Returns:
        pd.DataFrame: DataFrame with aggregated data based on domain ID.
    """
    subset_df = df[['tweet_id', 'author_id', 'tweet_metrics', 'context', 'entities']]

    subset_context_items = []
    for index, row in subset_df.iterrows():
        context_list = json.loads(row['context'])
        tweet_id     = row['tweet_id']
        author_id    = row['author_id']
        for item in context_list:
            domain_id   = item['domain']['id']
            domain_name = item['domain']['name']
            entity_id   = item['entity']['id']
            entity_name = item['entity']['name']
            data = {'tweet_id': tweet_id, 'author_id': author_id, 'domain_id':domain_id, 'domain_name': domain_name, 'entity_id':entity_id, 'entity_name': entity_name}
            subset_context_items.append(data)

    subset_context_item_df = pd.DataFrame(subset_context_items)
    return subset_context_item_df

subset_context_item_df = aggregate_data(df=tweets_df)

In [ ]:
def calculate_top_counts(data_frame, group_by_cols, aggregate_col, top_n):
    """
    Calculate the top counts based on the given DataFrame, group by columns, aggregate column, and number of top counts to retrieve.
    
    Parameters:
        data_frame (pandas.DataFrame): DataFrame containing the data to perform calculations on.
        group_by_cols (list): List of columns to group by.
        aggregate_col (str): Column to perform aggregation on.
        top_n (int): Number of top counts to retrieve.
        
    Returns:
        pandas.DataFrame: Top counts, sorted in descending order.
    """
    subset_agg = data_frame.groupby(by=group_by_cols).agg({aggregate_col: pd.Series.nunique}).reset_index().sort_values(by=aggregate_col, ascending=False)
    top_counts = subset_agg.head(top_n)
    
    return top_counts

top_20_domain_counts = calculate_top_counts(subset_context_item_df, ["domain_name"], "tweet_id", 20)
top_50_entity_counts = calculate_top_counts(subset_context_item_df, ["entity_name"], "tweet_id", 50)
top_25_user_counts = calculate_top_counts(subset_context_item_df, ["author_id"], "tweet_id", 25)

In [ ]:
fig, axs = plt.subplots(3, figsize = (25, 35))
fig.suptitle('Volume of Tweets by Domain & Entity (Context Annotations)')

sns.set_theme(style="whitegrid")
sns.set_color_codes("pastel")
sns.barplot(ax=axs[0], x=top_20_domain_counts['domain_name'], y=top_20_domain_counts['tweet_id'], color="b")
sns.barplot(ax=axs[1], x=top_50_entity_counts['tweet_id'], y=top_50_entity_counts['entity_name'], color="b", orient='h')
sns.barplot(ax=axs[2], x=top_25_user_counts['tweet_id'], y=top_25_user_counts['author_id'], color="b", orient='h')


# Rotate the x-axis labels
axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=60)
axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=60)
axs[2].set_xticklabels(axs[2].get_xticklabels(), rotation=60)


plt.tight_layout()  # Adjust spacing to avoid label overlap

plt.show()

In [ ]:
# Define the buckets for follower and following counts
buckets = [0, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000]

# Count the number of users in each bucket
follower_counts = pd.cut(users_df['followers_count'], buckets).value_counts().sort_index()
following_counts = pd.cut(users_df['following_count'], buckets).value_counts().sort_index()
# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 18))

# Plot follower count distribution
axs[0].bar(range(len(follower_counts)), follower_counts, width=0.4, align='center', alpha=0.5, color='blue')
axs[0].set_xlabel('Bucket')
axs[0].set_ylabel('User Count')
axs[0].set_title('Distribution of Follower Counts')
axs[0].set_xticks(range(len(follower_counts)))
axs[0].set_xticklabels(follower_counts.index, rotation=45)

# Plot following count distribution
axs[1].bar(range(len(following_counts)), following_counts, width=0.4, align='center', alpha=0.5, color='red')
axs[1].set_xlabel('Bucket')
axs[1].set_ylabel('User Count')
axs[1].set_title('Distribution of Following Counts')
axs[1].set_xticks(range(len(following_counts)))
axs[1].set_xticklabels(following_counts.index, rotation=45)

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Convert 'verified' column to integer type
users_df['verified'] = users_df['verified'].astype(int)

# Filter verified users
verified_users = users_df[users_df['verified'] == 1]
non_verified_users = users_df[users_df['verified'] == 0]

# Sort the verified users based on followers count in descending order
top_verified_followed_users = verified_users.sort_values('followers_count', ascending=False).head(25)

# Sort the verified users based on tweet count in descending order
top_verified_tweet_users = verified_users.sort_values('tweet_count', ascending=False).head(25)

# Sort the non-verified users based on followers count in descending order
top_non_verified_followed_users = non_verified_users.sort_values('followers_count', ascending=False).head(25)

# Sort the non-verified users based on tweet count in descending order
top_non_verified_tweet_users = non_verified_users.sort_values('tweet_count', ascending=False).head(25)

# Create a figure with a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(12, 12))

# Plot the top followed users (verified)
axs[0, 0].barh(top_verified_followed_users['username'], top_verified_followed_users['followers_count'], color='blue')
axs[0, 0].set_xlabel('Followers Count')
axs[0, 0].set_ylabel('Username')
axs[0, 0].set_title('Top 25 Most Followed Verified Users')
axs[0, 0].invert_yaxis()

# Plot the top tweet users (verified)
axs[0, 1].barh(top_verified_tweet_users['username'], top_verified_tweet_users['tweet_count'], color='green')
axs[0, 1].set_xlabel('Tweet Count')
axs[0, 1].set_ylabel('Username')
axs[0, 1].set_title('Top 25 Verified Users with Highest Tweet Counts')
axs[0, 1].invert_yaxis()

# Plot the top followed users (non-verified)
axs[1, 0].barh(top_non_verified_followed_users['username'], top_non_verified_followed_users['followers_count'], color='blue')
axs[1, 0].set_xlabel('Followers Count')
axs[1, 0].set_ylabel('Username')
axs[1, 0].set_title('Top 25 Most Followed Non-Verified Users')
axs[1, 0].invert_yaxis()

# Plot the top tweet users (non-verified)
axs[1, 1].barh(top_non_verified_tweet_users['username'], top_non_verified_tweet_users['tweet_count'], color='green')
axs[1, 1].set_xlabel('Tweet Count')
axs[1, 1].set_ylabel('Username')
axs[1, 1].set_title('Top 25 Non-Verified Users with Highest Tweet Counts')
axs[1, 1].invert_yaxis()

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

## Data Pre-Processing

The following code will remove emoticons, hyperlinks, whitepsaces such as new lines and indentations.
It will also tokenize text into words, remove slang words, and visualize the text pre-processed data to show patterns

In [ ]:
def removeUnicode(text):
    """ Removes unicode strings like "\u002c" and "x96" """
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    return text

def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [ ]:
def removeEmoticons(text):
    """ Removes emoticons from text """
    text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
    return text

def countEmoticons(text):
    """ Input: a text, Output: how many emoticons """
    return len(re.findall(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', text))


In [ ]:
nltk.download('stopwords')

""" Tokenizes a text to its words, removes and replaces some of them """    
finalTokens = [] # all tokens
stoplist = stopwords.words('english')
my_stopwords = "multiexclamation multiquestion multistop url atuser st rd nd th am pm" # my extra stopwords
stoplist = stoplist + my_stopwords.split()
allowedWordTypes = ["J","R","V","N"] #  J is Adject, R is Adverb, V is Verb, N is Noun. These are used for POS Tagging

# Create a DataFrame to store the preprocessed text and tweet ID
preprocessed_data = pd.DataFrame(columns=['tweet_id', 'text'])

In [ ]:
def tokenize(text, wordCountBefore, tweet_id):
    totalAdjectives = 0
    totalAdverbs = 0
    totalVerbs = 0
    onlyOneSentenceTokens = []  # tokens of one sentence each time

    tokens = nltk.word_tokenize(text)

    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)  # Technique 7: remove punctuation

    tokens = nltk.word_tokenize(text)

    tagged = nltk.pos_tag(tokens)  # Technique 13: part of speech tagging
    for w in tagged:
        if w[1][0] in allowedWordTypes and w[0] not in stoplist:
            onlyOneSentenceTokens.append(w[0])
            finalTokens.append(w[0])

    onlyOneSentence = " ".join(onlyOneSentenceTokens)

    # Store the preprocessed text and tweet ID in the DataFrame
    preprocessed_data.loc[len(preprocessed_data)] = [tweet_id, onlyOneSentence]

    return finalTokens

In [ ]:
from time import time
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Retrieve all tweets from the database
query = "SELECT DISTINCT * FROM tweets"
df_tweets = pd.read_sql_query(query, conn)
#Tweet_id is column 0, text is column 3


t0 = time()
totalSentences = 0
totalEmoticons = 0
totalSlangs = 0
totalSlangsFound = []
totalElongated = 0
totalMultiExclamationMarks = 0
totalMultiQuestionMarks = 0
totalMultiStopMarks = 0
totalAllCaps = 0

# Iterate over each row in df_tweets
for index, row in df_tweets.iterrows():
    totalSentences += 1
    feat = []

    tweet_id = row['tweet_id']
    text     = removeUnicode(row['text'])  # Technique 0
    text     = replaceURL(row['text'])

    wordCountBefore = len(re.findall(r'\w+', text))  # word count of one sentence before preprocess

    emoticons = countEmoticons(text)  # how many emoticons in this sentence
    totalEmoticons += emoticons

    text = removeEmoticons(text)  # removes emoticons from text
    tokens = tokenize(text, wordCountBefore, tweet_id)  

    # print("Processed tweet:", tweet_id)  # Print the tweet ID after processing

# View the resulting preprocessed data
print(preprocessed_data.head())
    
print("Total sentences: ",                          totalSentences,"\n")
print("Total Words before preprocess: ",            len(re.findall(r'\w+', ' '.join(df_tweets['text']))))
print("Total Distinct Tokens before preprocess: ",  len(set(re.findall(r'\w+', ' '.join(df_tweets['text'])))))
print("Average word/sentence before preprocess: ",  len(re.findall(r'\w+', ' '.join(df_tweets['text']))) / totalSentences, "\n")
print("Total Words after preprocess: ",             len(tokens))
print("Total Distinct Tokens after preprocess: ",   len(set(tokens)))
print("Average word/sentence after preprocess: ",   len(tokens)/totalSentences,"\n")


print("Total run time: ",                           time() - t0," seconds\n")

print("Total emoticons: ",                          totalEmoticons,"\n")
print("Total slangs: ",                             totalSlangs,"\n")
# commonSlangs = nltk.FreqDist(totalSlangsFound)
# for (word, count) in commonSlangs.most_common(20): # most common slangs across all texts
#     print(word,"\t",count)
# commonSlangs.plot(20, cumulative=False) # plot most common slangs

print("Total elongated words: ",                    totalElongated,"\n")
print("Total multi exclamation marks: ",            totalMultiExclamationMarks)
print("Total multi question marks: ",               totalMultiQuestionMarks)
print("Total multi stop marks: ",                   totalMultiStopMarks,"\n")
print("Total all capitalized words: ",              totalAllCaps,"\n")

#print(tokens)
commonWords = nltk.FreqDist(tokens)
print("Most common words ")
print("Word\tCount")
for (word, count) in commonWords.most_common(100): # most common words across all texts
    print(word,"\t",count)

# Create a larger plot with adjusted dimensions
plt.figure(figsize=(20, 8))  # Adjust the width and height as needed
commonWords.plot(100, cumulative=False) # plot most common words


bgm = nltk.collocations.BigramAssocMeasures()
tgm = nltk.collocations.TrigramAssocMeasures()
bgm_finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)
tgm_finder = nltk.collocations.TrigramCollocationFinder.from_words(tokens)
bgm_finder.apply_freq_filter(5) # bigrams that occur at least 5 times
print("Most common collocations (bigrams)")
print(bgm_finder.nbest(bgm.pmi, 50)) # top 50 bigram collocations
tgm_finder.apply_freq_filter(5) # trigrams that occur at least 5 times
print("Most common collocations (trigrams)")
print(tgm_finder.nbest(tgm.pmi, 20)) # top 20 trigrams collocations

In [ ]:
# Merge the preprocessed_data DataFrame with df_tweets based on tweet_id
df_tweets_preprocessed = df_tweets.merge(preprocessed_data, on='tweet_id', how='left', suffixes=('_original', '_preprocessed'))

# Deduplicate the merged DataFrame based on the 'tweet_id' column
df_tweets_preprocessed = df_tweets_preprocessed.drop_duplicates(subset=['tweet_id', 'text_preprocessed'])